# Problem Set 2 — Coding Part

**Course:** Data Compression With And Without Deep probabilistic models (Prof. Bamler at University of Tübingen)

- This notebook contains the coding part of Problem Set 2, published on 26 April 2023 and discussed on 3 May 2023.
- You can download the full problem set (including the pen-and-paper part) from the [course website](https://robamler.github.io/teaching/compress23/).


## Recap: Huffman *Encoder* From Last Problem Set

On the last problem set, you implemented the following Huffman encoder (this is given only for your reference; your task doesn't start until the next section below):

In [3]:
import heapq # See Problem 0.4 on Problem Set 0.

In [4]:
class HuffmanEncoder:
    """A code book for Huffman coding, optimized for *encoding*."""
    
    def __init__(self, probabilities):
        """Create a Huffman tree for symbols with the given `probabilities`.
        
        The alphabet is assumed to be {0, 1, 2, ..., len(probabilities) - 1}.

        Args:
            probabilities (list or numpy array): The probabilities of each
                symbol in the alphabet. The first entry is the probability of
                symbol `0`, the second entry is the probability of symbol `1`,
                and so on. All entries must be nonnegative and finite.
        """
        
        edges = [None] * (2 * len(probabilities) - 2)
        frontier = [(weight, index) for index, weight in enumerate(probabilities)]
        heapq.heapify(frontier)
        
        next_node = len(probabilities)
        while len(frontier) > 1:
            weight1, child1 = heapq.heappop(frontier)
            weight2, child2 = heapq.heappop(frontier)
            edges[child1] = (next_node, False)
            edges[child2] = (next_node, True)
            heapq.heappush(frontier, (weight1 + weight2, next_node))
            next_node += 1

        self.edges = edges
        
    def __getitem__(self, symbol):
        """Return the code word for a given `symbol`.

        Args:
            symbol (int): A symbol from the alphabet {0, 1, ..., len-1}.
                
        Returns:
            The code word for `symbol`.
        """
        assert 0 <= symbol and symbol < len(self)
        
        codeword_reverse = []
        node = symbol
        root_node = len(self.edges)
        while node != root_node:
            node, bit = self.edges[node]
            codeword_reverse.append(bit)
        
        return list(reversed(codeword_reverse))
    
    def __len__(self):
        """Returns the size of the alphabet."""
        # As discussed above, any binary tree with `N` leaves has
        # `M = 2*N - 2` edges. Therefore, `N = M / 2 + 1`.
        return len(self.edges) // 2 + 1 # ("//" denotes integer division)

#### Unit rests:

In [5]:
def minimal_encoder_test():
    probabilities = [.3, .28, .12, .1, .2]
    expected_codebook = {
        0: [True, True],
        1: [True, False],
        2: [False, True, True],
        3: [False, True, False],
        4: [False, False],
    }
    huffman_encoder = HuffmanEncoder(probabilities)
    for symbol, expected_codeword in expected_codebook.items():
        assert huffman_encoder[symbol] == expected_codeword

minimal_encoder_test()

## Problem 2.5: Implementing a Huffman *Decoder*

Let's now implement a new class `HuffmanDecoder` that constructs a Huffman tree that is optimized for *decoding*.
The above class `HuffmanEncoder` would be a bit awkward to use for decoding since we would always have to construct the entire codebook and then search for the one code word that matches the beginning of our compressed bit string (similar to how we implemented a generic decoder for prefix codes in Problem 0.3 (b) on Problem Set 0).

There's a smarter way to use a Huffman tree for decoding:
we can simply read the compressed bit string one bit at a time, and use these bits to follow the correspondingly labeled path from the root of the Huffman tree until we arrive at a leaf node.

For example, assume our Huffman tree looks as follows:

```text
symbols (leaves): 0   1   2   3
                   \   \ /   /
                    \   *   /
                     \ /   /
                      *   /
                       \ /
                       root
```

and assume that all "`\`" edges have label `False` and all "`/`" edges have label `True`.
Then, if we want to decode the compressed bit string `[False, True, False, True, ...]`,
we would start at the root node, turn left (for the first `False`), then turn right (`True`), and then turn left (`False`) to arrive at the leave node for symbol `1`.
Then we return to the root node, turn right, and arrive at the next symbol `3`, and so on.
This way, we never have to construct the code words for symbols that don't appear in the message.

In order to follow the above procedure, however, we have to store the Huffman tree in a different way than we did on the last problem set because we need to be able to follow edges *from parents to children* rather than the other way around.
The following is a proposal for how you could represent a Huffman tree that is optimized for *decoding* (as on the last problem set, feel free to use your own representation if you have a better idea):

Recall that we assume, without loss of generality, that the alphabet is $\mathfrak X = \{0,1,\ldots,|\mathfrak X|-1\}$ and that the nodes (or "vertices") are $V = \{0,1,\ldots, 2|\mathfrak X|-2\}$ with the first $|\mathfrak X|$ nodes representing the leaves (which correspond to the symbols in $\mathfrak X$) and the last node $(2|\mathfrak X|-2)$ representing the root node.
We again don't need to explicitly represent $V$ in memory as it is known implicitly.

We represent the edges of the tree in a list `edges` that, this time, is *indexed by the parent node* and *points to the node's two children*.
Since the first $|\mathfrak X|$ nodes in $V$ represent the leaves (which have no children), we shift the index into `edges` by `alphabet_size` $:= |\mathfrak X|$, i.e., we set `edges[node - alphabet_size] = (left_child, right_child)` where `node`, `left_child`, and `right_child` are all in $V$.
Here, `left_child` is the node that is linked to `node` by an edge with label `False`, and `right_child` is the node that is linked to `node` by an edge with label `True`.

### Your Tasks:

- Read and understand the code in the method `decode` below.
  Then implement the body of the constructor (`__init__`) so that it creates the tree structure that `decode` expects (as described above).
- Run the minimal provided unit test to make sure you didn't make any obvious mistake.
- Implement and run additional unit tests. In particular, test edge cases like ties or a degenerate probability distribution (that puts all mass on a single symbol.
- Finally, implement a more extensive round-trip test where you encode a long random message using your `HuffmanEncoder`, decode it with your `HuffmanDecoder`, and then verify that the decoded message matches the original message.

In [6]:
class HuffmanDecoder:
    """A code book for Huffman coding, optimized for *decoding*."""
    
    def __init__(self, probabilities):
        """Create a Huffman tree for symbols with the given `probabilities`.
        
        The alphabet is assumed to be {0, 1, 2, ..., len(probabilities) - 1}.

        Args:
            probabilities (list or numpy array): The probabilities of each
                symbol in the alphabet. The first entry is the probability of
                symbol `0`, the second entry is the probability of symbol `1`,
                and so on. All entries must be nonnegative and finite.
        """
        
        # TODO: implement the constructor; it will look similar to the
        # construtor of `HuffmanEncoder`.

    def decode(self, encoded):
        """Decode a sequence of symbols.
        
        Args:
            encoded: A compressed representation of a message, represented
                as a list of boolean values
            
        Yields:
            A sequence of decoded symbols. Thus, you can iterate over the
            resulting symbols as follows:

            ```
            compressed = [True, False, False, ...]
            huffman_decoder = HuffmanDecoder(...)
            for symbol in huffman_decoder.decode(compressed):
                # ... do something with `symbol` ...
            ```
        """

        alphabet_size = len(self)
        root_node = 2 * alphabet_size - 2
        node = root_node
        for bit in encoded:
            left_child, right_child = self.edges[node - alphabet_size]
            node = right_child if bit else left_child
            if node < alphabet_size:
                # Reached a leaf node; thus, `node` is the decoded symbol.
                # (`yield` is similar to `return` but can be used multiple times.)
                yield node
                node = root_node

    def __len__(self):
        """Returns the size of the alphabet."""
        # Any binary tree with `N` leaves has `M = N - 1` non-leaf nodes.
        # We thus have an alphabet size of `N = M + 1`, where
        # `M = len(self.edges)` since `self.edges` contains one entry per
        # non-leaf node
        return len(self.edges) + 1

#### Unit rests:

In [7]:
def minimal_decoder_test():
    probabilities = [.3, .28, .12, .1, .2]
    expected_codebook = {
        0: [True, True],
        1: [True, False],
        2: [False, True, True],
        3: [False, True, False],
        4: [False, False],
    }
    encoded = [True, False, False, True, True, False, False, True, True]
    huffman_decoder = HuffmanDecoder(probabilities)
    decoded = list(huffman_decoder.decode(encoded))
    assert decoded == [1, 2, 4, 0]

minimal_decoder_test()

In [ ]:
# TODO: implement a round-trip test.